In [2]:
import math
from collections import defaultdict
import random

random.seed(0)

In [4]:
def sigmoid(x):
    return 1.0 / (1.0 + math.exp(-x))

def d_sigmoid(x):
    s = sigmoid(x)
    return s * (1.0 - s)

In [5]:
# --- Tiny network: 2 -> 2 -> 2 -> 2 -> 1 (3 hidden layers) ---
# Weights (Wl: shape [n_l, n_{l-1}]), biases (bl: shape [n_l])
W1 = [[0.3, -0.2],
      [0.1,  0.4]]
b1 = [0.0, 0.1]

W2 = [[-0.5, 0.2],
      [ 0.3, 0.7]]
b2 = [0.2, -0.3]

W3 = [[ 0.6, -0.1],
      [-0.4,  0.2]]
b3 = [-0.2, 0.05]

W4 = [[0.8, -0.6]]  # output layer: 1 x 2
b4 = [0.15]

# Single data point (supervised)
x = [0.7, -1.2]
y_true = 0.9

In [6]:
# --- Forward pass (store z and a per layer) ---
def matvec(W, v):
    return [sum(W[i][j]*v[j] for j in range(len(v))) for i in range(len(W))]

def vecadd(a, b):
    return [a[i] + b[i] for i in range(len(a))]

In [8]:
# layer 1
z1 = vecadd(matvec(W1, x), b1)
a1 = [sigmoid(z) for z in z1]

# layer 2
z2 = vecadd(matvec(W2, a1), b2)
a2 = [sigmoid(z) for z in z2]

# layer 3
z3 = vecadd(matvec(W3, a2), b3)
a3 = [sigmoid(z) for z in z3]

# output layer
z4 = vecadd(matvec(W4, a3), b4)  # 1-dim list
y_hat = sigmoid(z4[0])

# Loss: 0.5 * (y - y_hat)^2
L = 0.5 * (y_true - y_hat)**2

L

0.055878068519147776

In [9]:
call_counter = defaultdict(int)


defaultdict(int, {})

In [10]:
# Map layer index to activations and pre-activations
# Layers: 1..4 (4 is output layer)
Z = {1: z1, 2: z2, 3: z3, 4: z4}
A = {0: x, 1: a1, 2: a2, 3: a3, 4: [y_hat]}  # A[0] is input; A[4] is post-activation output
W = {1: W1, 2: W2, 3: W3, 4: W4}
b = {1: b1, 2: b2, 3: b3, 4: b4}


In [11]:
# Useful sizes
sizes = {0: len(x), 1: len(a1), 2: len(a2), 3: len(a3), 4: 1}

# Derivative of loss wrt pre-activation z_l[i], computed naively with recursion and NO caching.
def dL_dz_naive(l, i):
    call_counter[(l, i)] += 1
    if l == 4:
        # dL/dz4 = (y_hat - y_true) * sigmoid'(z4)
        return (y_hat - y_true) * d_sigmoid(Z[4][0])
    else:
        # dL/da_l[i] = sum_m dL/dz_{l+1}[m] * d z_{l+1}[m] / d a_l[i]
        # z_{l+1}[m] = sum_k W_{l+1}[m,k] * a_l[k] + b_{l+1}[m]
        # => derivative wrt a_l[i] is W_{l+1}[m, i]
        s = 0.0
        for m in range(sizes[l+1]):
            # Critically, this recursively recomputes dL/dz_{l+1}[m] again and again for different i.
            s += dL_dz_naive(l+1, m) * W[l+1][m][i]
        # Chain by activation derivative at layer l
        return s * d_sigmoid(Z[l][i])

# Gradients wrt weights and biases using the naive dL_dz
def grad_W_naive(l, i, j):
    # dL/dW_l[i,j] = dL/dz_l[i] * d z_l[i] / d W_l[i,j] = dL/dz_l[i] * a_{l-1}[j]
    return dL_dz_naive(l, i) * A[l-1][j]

def grad_b_naive(l, i):
    # dL/db_l[i] = dL/dz_l[i] * d z_l[i] / d b_l[i] = dL/dz_l[i]
    return dL_dz_naive(l, i)

In [15]:
# Compute all gradients (this will trigger tons of redundant recursive calls)
grads_W = {l: [[0.0]*len(W[l][0]) for _ in range(len(W[l]))] for l in [1,2,3,4]}
grads_b = {l: [0.0]*len(b[l]) for l in [1,2,3,4]}

for l in [1,2,3,4]:
    for i in range(len(W[l])):
        for j in range(len(W[l][0])):
            grads_W[l][i][j] = grad_W_naive(l, i, j)
    for i in range(len(b[l])):
        grads_b[l][i] = grad_b_naive(l, i)

In [16]:
grads_W

{1: [[0.0003582243755061708, -0.0006140989294391499],
  [-4.473004029712163e-06, 7.668006908077993e-06]],
 2: [[-0.0022556775919990012, -0.001562969397063531],
  [0.000621642245487877, 0.0004307387762620629]],
 3: [[-0.008124538649331792, -0.008943799091209098],
  [0.006093830245667664, 0.006708318560053563]],
 4: [[-0.041936867458953586, -0.04026553028316163]]}

In [17]:
grads_b

{1: [0.0005117491078659583, -6.390005756731661e-06],
 2: [-0.0036939611256399767, 0.0010180188414483206],
 3: [-0.016418980336164235, 0.012315096683522982],
 4: [-0.0821318445040607]}

In [18]:
# Display results
print("Forward pass:")
print(f"  x    = {x}")
print(f"  z1   = {z1}\n  a1   = {a1}")
print(f"  z2   = {z2}\n  a2   = {a2}")
print(f"  z3   = {z3}\n  a3   = {a3}")
print(f"  z4   = {z4}\n  y_hat= {y_hat:.6f}")
print(f"  Loss = {L:.8f}\n")

Forward pass:
  x    = [0.7, -1.2]
  z1   = [0.44999999999999996, -0.30999999999999994]
  a1   = [0.610639233949222, 0.42311473886795364]
  z2   = [-0.020696669201020257, 0.17937208739233418]
  a2   = [0.49482601738895976, 0.5447231745268372]
  z3   = [0.04242329298069214, -0.038985772050216486]
  a3   = [0.5106042328914234, 0.49025479126029914]
  z4   = [0.2643305115569593]
  y_hat= 0.565701
  Loss = 0.05587807



In [19]:
print("Naive (no-backprop) gradients:")
for l in [1,2,3,4]:
    print(f"W{l} grads:")
    for row in grads_W[l]:
        print("  ", [f"{v:.8f}" for v in row])
    print(f"b{l} grads:")
    print("  ", [f"{v:.8f}" for v in grads_b[l]])
    print()

# Show how many times each dL/dz_l[i] was recomputed
total_calls = sum(call_counter.values())
print(f"Total dL/dz naive calls (showing blow-up due to no reuse): {total_calls}")
for key in sorted(call_counter.keys()):
    print(f"  calls dL/dz for layer {key[0]} neuron {key[1]}: {call_counter[key]}")

Naive (no-backprop) gradients:
W1 grads:
   ['0.00035822', '-0.00061410']
   ['-0.00000447', '0.00000767']
b1 grads:
   ['0.00051175', '-0.00000639']

W2 grads:
   ['-0.00225568', '-0.00156297']
   ['0.00062164', '0.00043074']
b2 grads:
   ['-0.00369396', '0.00101802']

W3 grads:
   ['-0.00812454', '-0.00894380']
   ['0.00609383', '0.00670832']
b3 grads:
   ['-0.01641898', '0.01231510']

W4 grads:
   ['-0.04193687', '-0.04026553']
b4 grads:
   ['-0.08213184']

Total dL/dz naive calls (showing blow-up due to no reuse): 111
  calls dL/dz for layer 1 neuron 0: 3
  calls dL/dz for layer 1 neuron 1: 3
  calls dL/dz for layer 2 neuron 0: 9
  calls dL/dz for layer 2 neuron 1: 9
  calls dL/dz for layer 3 neuron 0: 21
  calls dL/dz for layer 3 neuron 1: 21
  calls dL/dz for layer 4 neuron 0: 45
